# Many Bhabha events in MCParticlesSkimmed

In [ ]:
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import uproot

from lcio_checks.mc.simulation import add_simulation_info
from lcio_checks.util import config, load_or_make

f = uproot.open(f"{config['data_dir']}/P2f_z_eehiq.root")["MyLCTuple"]
mc = f.arrays(filter_name="mc*", entry_stop=-1)
mc = add_simulation_info(mc)

In [ ]:
@load_or_make(["energy_sum_per_generator_status"])
def energy_sum_per_generator_status():
    fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(10, 6))
    for gst, ax in zip(range(5), axs.flatten()):
        bins = ax.hist(
            ak.sum(mc[(mc.mcgst == gst)].mcene, axis=1), bins=500, label="all"
        )[1]
        ax.hist(
            ak.sum(mc[(mc.mcgst == gst) & ~mc.isOverlay].mcene, axis=1),
            bins=bins,
            alpha=0.8,
            label="not isOverlay",
        )
        ax.legend()
        ax.set_title(f"Generator status {gst}")
        ax.set_xlabel("energy sum in event [GeV]")
        ax.set_yscale("log")
    fig.tight_layout()
    return (fig,)


energy_sum_per_generator_status();

## Energy in event after angular selection cut


In [ ]:
@load_or_make(["energy_sum_angular_acceptance_daughterless"])
def energy_sum_angular_acceptance_daughterless():
    bins = np.arange(0, 1500, 10)
    mcps = mc[mc.mcda0 == -1]
    mcps = mcps[mcps.mcene > 0]
    fig, ax = plt.subplots(figsize=(7, 5))
    for cos_theta in [0.999999, 0.99999, 0.9999, 0.999, 0.99, 0.9]:
        x = ak.sum(mcps[np.abs(mcps.mcmoz) / mcps.mcene < cos_theta].mcene, axis=1)
        ax.hist(x, bins=bins, label=str(cos_theta), alpha=0.7)
    ax.legend(title="|cos$\\theta$|$\\leq$")
    ax.set_yscale("log")
    ax.set_title("All daughter-less generated particles")
    ax.set_xlabel("energy in event [GeV]")
    return (fig,)


energy_sum_angular_acceptance_daughterless()

In [ ]:
@load_or_make(["energy_sum_angular_acceptance_undecayed_in_detector"])
def energy_sum_angular_acceptance_undecayed_in_detector():
    bins = np.arange(0, 1500, 10)
    mcps = mc[mc.mcgst == 1]
    mcps = mcps[mcps.mcene > 0]
    fig, ax = plt.subplots(figsize=(7, 5))
    for cos_theta in [0.999999, 0.99999, 0.9999, 0.999, 0.99, 0.9]:
        x = ak.sum(mcps[np.abs(mcps.mcmoz) / mcps.mcene < cos_theta].mcene, axis=1)
        ax.hist(x, bins=bins, label=str(cos_theta), alpha=0.7)
    ax.legend(title="|cos$\\theta$|$\\leq$")
    ax.set_yscale("log")
    ax.set_title("Generator status 1 (undecayed in detector)")
    ax.set_xlabel("energy in event [GeV]")
    return (fig,)


energy_sum_angular_acceptance_undecayed_in_detector();

## Generator status 1

In [ ]:
@load_or_make(["overlay_vs_not"])
def overlay_vs_not(mcps):
    fig, axs = plt.subplots(figsize=(10, 5), ncols=2)

    x = ak.num(mcps[~mcps.isOverlay].mcene)
    axs[0].hist(x, bins=np.arange(ak.max(x) + 1) - 0.5)
    axs[0].set_title("Generator status 1, #MCP not overlay")

    x = ak.num(mcps[mcps.isOverlay].mcene)
    axs[1].hist(x, bins=np.arange(ak.max(x) + 1) - 0.5)
    axs[1].set_title("Generator status 1, #overlay MCP")
    return (fig,)


overlay_vs_not(mc[mc.mcgst == 1]);

In [ ]:
is_vars = [is_var for is_var in mc.fields if is_var.startswith("is")]
df = ak.to_pandas(mc[mc.mcgst == 1][is_vars])
df.describe()

### Simulation stati of the MCP in one event (generator status 1)

In [ ]:
df.loc[0]